In [ ]:
import os
import uuid
import json
from pprint import pprint
from langchain_core.messages import HumanMessage, AIMessage, ToolCall, ToolMessage
from rich import print as rprint

from utils import authenticate_gmail, check_emails_from_sender, get_email_details, parse_email_content
from definitions import app

In [2]:
tool_names = [
    'invoice_api_tool', 
    'bol_api_tool', 
    'shipping_label_api_tool', 
    'item_label_api_tool',
    'receipt_api_tool',
    'extract_structured_text_tool'
]

In [ ]:
def run_agentic_workflow(email_data, email_thread_id):

    content = {
        "subject": email_data.get('subject') or None,
        "sender": email_data.get('sender') or None,
        "body": email_data.get('body') or None,
        "has_attachments": email_data.get('has_attachments'),
    }

    if email_data.get('has_attachments'):
        absolute_attachment_paths = [
            "/home/hamzajadoon/Work/PackageX/Agents/email_agent_workflow/" + path
            for path in email_data['attachment_paths']
        ]
        content["attachment_paths"] = absolute_attachment_paths
    else:
        content["attachment_paths"] = None

    email_summary = f"""
    Email Subject:
    {content['subject']}

    Email Body:
    {content['body']}

    ### Attachment Status
    has_attachments: {content["has_attachments"]}
    
    Attached Files:
    {content["attachment_paths"]}
    """

    workflow_input = {
        "messages": [
            {
                "role": "user",
                "content": email_summary
            }
        ]
    }

    config = {
        "metadata": {
            "langchain_project": os.environ.get("LANGSMITH_PROJECT")
        }
    }

    print("Streaming response...\n")
    seen_message_ids = set()
    for chunk in app.stream(workflow_input, config):
        for node_name, state in chunk.items():
            messages = state.get("messages", [])
            if messages:

                # print(len(messages))
                # rprint(messages)
                # print("\n\n-------------------------------------------\n\n")
                
                for msg in messages:
                    if isinstance(msg, AIMessage) and msg.id not in seen_message_ids:
                        seen_message_ids.add(msg.id)

                        if hasattr(msg, "tool_calls") and msg.tool_calls:
                            print(f"\nAgent: {msg.name}")
                            if len(msg.content) != 0: print(f"  Reasoning: {msg.content}")
                            print(f"  Handoff Tool: {msg.tool_calls[0]['name']}")
                            print(f"  Arguments for selected subagent/tool\n    {msg.tool_calls[0]['args'].keys()}")
                            # pprint(msg.tool_calls[0]['args'])
                        
                        # if hasattr(msg, "tool_calls") and msg.tool_calls:
                        #     print(f"Agent: {msg.name}")
                        #     print(f"  Handoff: {msg.tool_calls[0]['name']}")
                        #     print(f"  Arguments for selected subagent\n    {msg.tool_calls[0]['args'].keys()}")

                    if isinstance(msg, ToolMessage) and msg.id not in seen_message_ids and msg.name in tool_names:
                        seen_message_ids.add(msg.id)
                        print(f"\nTool: {msg.name}")
                        # print("  Output:")
                        # try:
                        #     pprint(json.loads(msg.content))
                        # except: 
                        #     print("could not parse into json")
                        #     print(msg.content)                   
                        
    # print("\n Final Output")
    # try:
    #     pprint(json.loads(messages[-1].content))
    # except:
    #     print("could not parse into json")

    print("\nWorkflow finished for email")
    return messages

In [ ]:
# TARGET_SENDER_EMAIL = "masterblaster012361@gmail.com"
# TARGET_SENDER_EMAIL = "muhammadhamzajadoon@gmail.com"
# TARGET_SENDER_EMAIL = "muhammadhamzaj2001@gmail.com"
# TARGET_SENDER_EMAIL = "humzaahmad9066@gmail.com"
TARGET_SENDER_EMAIL = "zain@pkgx.io"

gmail_service = authenticate_gmail()
processed_emails = []
if gmail_service:
  processed_emails.append(check_emails_from_sender(gmail_service, TARGET_SENDER_EMAIL, mark_as_read=True))

In [ ]:
email_thread_id = str(uuid.uuid4())
workflow_output = run_agentic_workflow(processed_emails[0][1], email_thread_id)

In [10]:
results = []
for i in processed_emails[0]:
    pprint(i)
    email_thread_id = str(uuid.uuid4())
    workflow_output = run_agentic_workflow(i, email_thread_id)
    results.append({"input": i, "output": workflow_output})
    print('-----------------------------------------')

{'attachment_paths': [],
 'body': 'Dear Logistics Team,\r\n'
         '\r\n'
         'I hope this message finds you well.\r\n'
         '\r\n'
         'We would like to inform you that the delivery of your shipment, '
         'originally\r\n'
         'scheduled to arrive on 12 June 2025, has been delayed due to a '
         'mechanical\r\n'
         'issue with the linehaul vehicle currently en route from Manchester '
         'to\r\n'
         'London. The affected shipment is referenced under REF# 78452-LDN, '
         'with BOL\r\n'
         'number BOL-20250607-223 and PO number PO-31489-MN.\r\n'
         '\r\n'
         'Our team is actively working with the carrier to resolve the issue '
         'and\r\n'
         'arrange for alternative transport. We will share the revised '
         'estimated\r\n'
         'delivery date as soon as it is confirmed.\r\n'
         '\r\n'
         'We apologize for the inconvenience caused and appreciate your '
         'patience.\r\n'
    

In [17]:
for i in results:
    pprint(i['input'])
    rprint(i['output'])
    rprint(i['output'][-1].content)
    print("-------------------------------------------------------------")

{'attachment_paths': [],
 'body': 'Dear Logistics Team,\r\n'
         '\r\n'
         'I hope this message finds you well.\r\n'
         '\r\n'
         'We would like to inform you that the delivery of your shipment, '
         'originally\r\n'
         'scheduled to arrive on 12 June 2025, has been delayed due to a '
         'mechanical\r\n'
         'issue with the linehaul vehicle currently en route from Manchester '
         'to\r\n'
         'London. The affected shipment is referenced under REF# 78452-LDN, '
         'with BOL\r\n'
         'number BOL-20250607-223 and PO number PO-31489-MN.\r\n'
         '\r\n'
         'Our team is actively working with the carrier to resolve the issue '
         'and\r\n'
         'arrange for alternative transport. We will share the revised '
         'estimated\r\n'
         'delivery date as soon as it is confirmed.\r\n'
         '\r\n'
         'We apologize for the inconvenience caused and appreciate your '
         'patience.\r\n'
    

[
    HumanMessage(
        content='\n    Email Subject:\n    Delay Notification for Shipment REF# 78452-LDN\n\n    Email Body:\n    
Dear Logistics Team,\r\n\r\nI hope this message finds you well.\r\n\r\nWe would like to inform you that the 
delivery of your shipment, originally\r\nscheduled to arrive on 12 June 2025, has been delayed due to a 
mechanical\r\nissue with the linehaul vehicle currently en route from Manchester to\r\nLondon. The affected 
shipment is referenced under REF# 78452-LDN, with BOL\r\nnumber BOL-20250607-223 and PO number 
PO-31489-MN.\r\n\r\nOur team is actively working with the carrier to resolve the issue and\r\narrange for 
alternative transport. We will share the revised estimated\r\ndelivery date as soon as it is confirmed.\r\n\r\nWe 
apologize for the inconvenience caused and appreciate your patience.\r\nPlease don’t hesitate to reach out if you 
have any further questions or\r\nrequire support in the meantime.\r\n\r\nBest regards,\r\n*Karen Willis*\r\nCarrier
Operations Coordinator\r\nTransway Freight Services\r\nkaren.willis@transwayfreight.com\r\n+44 20 7946 1123\r\n\n\n
### Attachment Status\n    has_attachments: False\n    \n    Attached Files:\n    None\n    ',
        additional_kwargs={},
        response_metadata={},
        id='0cd5c52f-7474-483a-a156-ad3a9e27dbf5'
    ),
    AIMessage(
        content='',
        additional_kwargs={
            'tool_calls': [
                {
                    'id': 'call_wjSunPZEa8mhOZu1XdE7JOcq',
                    'function': {
                        'arguments': '\n{\n  "email_body": "Dear Logistics Team,\\n\\nI hope this message finds you
well.\\n\\nWe would like to inform you that the delivery of your shipment, originally\\nscheduled to arrive on 12 
June 2025, has been delayed due to a mechanical\\nissue with the linehaul vehicle currently en route from 
Manchester to\\nLondon. The affected shipment is referenced under REF# 78452-LDN, with BOL\\nnumber 
BOL-20250607-223 and PO number PO-31489-MN.\\n\\nOur team is actively working with the carrier to resolve the issue
and\\narrange for alternative transport. We will share the revised estimated\\ndelivery date as soon as it is 
confirmed.\\n\\nWe apologize for the inconvenience caused and appreciate your patience.\\nPlease don’t hesitate to 
reach out if you have any further questions or\\nrequire support in the meantime.\\n\\nBest regards,\\n*Karen 
Willis*\\nCarrier Operations Coordinator\\nTransway Freight Services\\nkaren.willis@transwayfreight.com\\n+44 20 
7946 1123"\n}',
                        'name': 'transfer_to_text_extractor_agent'
                    },
                    'type': 'function'
                }
            ],
            'refusal': None
        },
        response_metadata={
            'token_usage': {
                'completion_tokens': 236,
                'prompt_tokens': 1547,
                'total_tokens': 1783,
                'completion_tokens_details': {
                    'accepted_prediction_tokens': 0,
                    'audio_tokens': 0,
                    'reasoning_tokens': 0,
                    'rejected_prediction_tokens': 0
                },
                'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}
            },
            'model_name': 'gpt-4-0613',
            'system_fingerprint': None,
            'id': 'chatcmpl-BhBu8yAjYyw0scibhvfYaBwHH3SgD',
            'service_tier': 'default',
            'finish_reason': 'tool_calls',
            'logprobs': None
        },
        name='supervisor',
        id='run--9d711a49-67c9-432b-927f-8a3b4d1512f7-0',
        tool_calls=[
            {
                'name': 'transfer_to_text_extractor_agent',
                'args': {
                    'email_body': 'Dear Logistics Team,\n\nI hope this message finds you well.\n\nWe would like to 
inform you that the delivery of your shipment, originally\nscheduled to arrive on 12 June 2025, has been delayed 
due to a mechani

{
    "email_summary": "Shipment delay notification for REF# 78452-LDN originally scheduled for 12 June 2025 due to a
mechanical issue with the vehicle.",
    "processing_intent": "text_data_extraction",
    "extracted_data": {
        "Message": {
            "Greeting": "Dear Logistics Team",
            "Body": {
                "Introduction": "I hope this message finds you well.",
                "Notification": {
                    "ShipmentDelay": {
                        "OriginalDeliveryDate": "2025-06-12",
                        "Reason": "mechanical issue with the linehaul vehicle",
                        "Route": "Manchester to London",
                        "ShipmentDetails": {
                            "ReferenceNumber": "78452-LDN",
                            "BOLNumber": "BOL-20250607-223",
                            "PONumber": "PO-31489-MN"
                        },
                        "CurrentStatus": "Our team is actively working with the carrier to resolve the issue and 
arrange for alternative transport. We will share the revised estimated delivery date as soon as it is confirmed."
                    },
                    "Apology": "We apologize for the inconvenience caused and appreciate your patience. Please 
don’t hesitate to reach out if you have any further questions or require support in the meantime."
                }
            },
            "Closing": "Best regards",
            "Sender": {
                "Name": "Karen Willis",
                "Role": "Carrier Operations Coordinator",
                "Company": "Transway Freight Services",
                "Email": "karen.willis@transwayfreight.com",
                "Phone": "+44 20 7946 1123"
            }
        }
    }
}

-------------------------------------------------------------
{'attachment_paths': ['./Invoice 182407-1.jpg'],
 'body': 'Dear Logistics Team,\r\n'
         '\r\n'
         'I hope this message finds you well.\r\n'
         '\r\n'
         'We would like to inform you that the delivery of your shipment, '
         'originally\r\n'
         'scheduled to arrive on 12 June 2025, has been delayed due to a '
         'mechanical\r\n'
         'issue with the linehaul vehicle currently en route from Manchester '
         'to\r\n'
         'London. The affected shipment is referenced under REF# 78452-LDN, '
         'with BOL\r\n'
         'number BOL-20250607-223 and PO number PO-31489-MN.\r\n'
         '\r\n'
         'Our team is actively working with the carrier to resolve the issue '
         'and\r\n'
         'arrange for alternative transport. We will share the revised '
         'estimated\r\n'
         'delivery date as soon as it is confirmed.\r\n'
         '\r\n'
         'We apol

[
    HumanMessage(
        content="\n    Email Subject:\n    Delay Notification for Shipment REF# 78452-LDN\n\n    Email Body:\n    
Dear Logistics Team,\r\n\r\nI hope this message finds you well.\r\n\r\nWe would like to inform you that the 
delivery of your shipment, originally\r\nscheduled to arrive on 12 June 2025, has been delayed due to a 
mechanical\r\nissue with the linehaul vehicle currently en route from Manchester to\r\nLondon. The affected 
shipment is referenced under REF# 78452-LDN, with BOL\r\nnumber BOL-20250607-223 and PO number 
PO-31489-MN.\r\n\r\nOur team is actively working with the carrier to resolve the issue and\r\narrange for 
alternative transport. We will share the revised estimated\r\ndelivery date as soon as it is confirmed.\r\n\r\nWe 
apologize for the inconvenience caused and appreciate your patience.\r\nPlease don’t hesitate to reach out if you 
have any further questions or\r\nrequire support in the meantime.\r\n\r\nBest regards,\r\n*Karen Willis*\r\nCarrier
Operations Coordinator\r\nTransway Freight Services\r\nkaren.willis@transwayfreight.com\r\n+44 20 7946 1123\r\n\n\n
### Attachment Status\n    has_attachments: True\n    \n    Attached Files:\n    
['/home/hamzajadoon/Work/PackageX/Agents/email_agent_workflow/./Invoice 182407-1.jpg']\n    ",
        additional_kwargs={},
        response_metadata={},
        id='a18d114a-a175-4721-a4ea-548163bc0237'
    ),
    AIMessage(
        content='',
        additional_kwargs={
            'tool_calls': [
                {
                    'id': 'call_rdnSnHQlWYwgCitnoAhysr8F',
                    'function': {'arguments': '{}', 'name': 'transfer_to_acknowledgment_agent'},
                    'type': 'function'
                }
            ],
            'refusal': None
        },
        response_metadata={
            'token_usage': {
                'completion_tokens': 11,
                'prompt_tokens': 1573,
                'total_tokens': 1584,
                'completion_tokens_details': {
                    'accepted_prediction_tokens': 0,
                    'audio_tokens': 0,
                    'reasoning_tokens': 0,
                    'rejected_prediction_tokens': 0
                },
                'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}
            },
            'model_name': 'gpt-4-0613',
            'system_fingerprint': None,
            'id': 'chatcmpl-BhBv49Lzd6SjO8PyqI3sBOB6OQHTd',
            'service_tier': 'default',
            'finish_reason': 'tool_calls',
            'logprobs': None
        },
        name='supervisor',
        id='run--3d1de10c-ad37-4aa0-a80f-b89626e0d3b7-0',
        tool_calls=[
            {
                'name': 'transfer_to_acknowledgment_agent',
                'args': {},
                'id': 'call_rdnSnHQlWYwgCitnoAhysr8F',
                'type': 'tool_call'
            }
        ],
        usage_metadata={
            'input_tokens': 1573,
            'output_tokens': 11,
            'total_tokens': 1584,
            'input_token_details': {'audio': 0, 'cache_read': 0},
            'output_token_details': {'audio': 0, 'reasoning': 0}
        }
    ),
    ToolMessage(
        content='Successfully transferred to acknowledgment_agent',
        name='transfer_to_acknowledgment_agent',
        id='78c936d9-b3e5-4fb1-80b1-46b8ab832327',
        tool_call_id='call_rdnSnHQlWYwgCitnoAhysr8F'
    ),
    AIMessage(
        content='**Email Summary**: The sender, Karen Willis from Transway Freight Services, is notifying about a 
delay in the delivery of shipment REF# 78452-LDN due to a mechanical issue with the vehicle. \n\n**Relevance 
Assessment**: The email is logistics-related as it pertains to a delay in shipment delivery.\n\n**Intent 
Classification**: This is a business confirmation email, informing about a change in the delivery 
schedule.\n\n**Return Acknowledgment**:\n\n{\n    "email_summary": "Notification of shipment delay for REF# 
78452-LDN due to vehicle

...

The extracted data from the attached Bill of Lading is as follows:

- Sender Business: Evident Olympus
- Recipient Name: Joey Dooling
- Recipient Address: Freepor, TX
- Shipping Date: August 22, 2023

Please note that the freight charges and delivery terms were not found in the document.

-------------------------------------------------------------
{'attachment_paths': ['./102-ghj-company_png.rf.320a2b55c06bdd1b43ad1404ed4db231.jpg'],
 'body': 'Hi Sarah,\r\n'
         '\r\n'
         'Please process the attached commercial invoice for our new shipment '
         'from\r\n'
         'China. We need the vendor details, line items, and total amount '
         'extracted\r\n'
         'for our ERP system. The data should be ready by EOD for customs '
         'clearance.\r\n'
         '\r\n'
         'Thanks!\r\n',
 'date': 'Wed, 11 Jun 2025 11:17:20 +0500',
 'has_attachments': True,
 'message_id': '1975da2fdacc1033',
 'sender': 'Muhammad Hamza Jadoon <muhammadhamzajadoon@gmail.com>',
 'subject': 'Invoice Processing Request - Order #LG-4567'}


[
    HumanMessage(
        content="\n    Email Subject:\n    Invoice Processing Request - Order #LG-4567\n\n    Email Body:\n    Hi 
Sarah,\r\n\r\nPlease process the attached commercial invoice for our new shipment from\r\nChina. We need the vendor
details, line items, and total amount extracted\r\nfor our ERP system. The data should be ready by EOD for customs 
clearance.\r\n\r\nThanks!\r\n\n\n    ### Attachment Status\n    has_attachments: True\n    \n    Attached Files:\n 
['/home/hamzajadoon/Work/PackageX/Agents/email_agent_workflow/./102-ghj-company_png.rf.320a2b55c06bdd1b43ad1404ed4d
b231.jpg']\n    ",
        additional_kwargs={},
        response_metadata={},
        id='f4c055c0-5913-42b2-a7f4-30d559f43c56'
    ),
    AIMessage(
        content='',
        additional_kwargs={
            'tool_calls': [
                {
                    'id': 'call_7E4Do97RVGwJpyvcFzIrHWbK',
                    'function': {
                        'arguments': '\n{\n  "email_body": "Hi Sarah,\\n\\nPlease process the attached commercial 
invoice for our new shipment from\\nChina. We need the vendor details, line items, and total amount extracted\\nfor
our ERP system. The data should be ready by EOD for customs clearance.\\n\\nThanks!",\n  "image_paths": 
["/home/hamzajadoon/Work/PackageX/Agents/email_agent_workflow/./102-ghj-company_png.rf.320a2b55c06bdd1b43ad1404ed4d
b231.jpg"],\n  "document_type": "invoice"\n}',
                        'name': 'transfer_to_document_processor_agent'
                    },
                    'type': 'function'
                }
            ],
            'refusal': None
        },
        response_metadata={
            'token_usage': {
                'completion_tokens': 134,
                'prompt_tokens': 1438,
                'total_tokens': 1572,
                'completion_tokens_details': {
                    'accepted_prediction_tokens': 0,
                    'audio_tokens': 0,
                    'reasoning_tokens': 0,
                    'rejected_prediction_tokens': 0
                },
                'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}
            },
            'model_name': 'gpt-4-0613',
            'system_fingerprint': None,
            'id': 'chatcmpl-BhC12eOuYygTsshIG1wG3Nc5Sc4Bz',
            'service_tier': 'default',
            'finish_reason': 'tool_calls',
            'logprobs': None
        },
        name='supervisor',
        id='run--56421ede-49c7-4da4-9fbe-ea64c552581b-0',
        tool_calls=[
            {
                'name': 'transfer_to_document_processor_agent',
                'args': {
                    'email_body': 'Hi Sarah,\n\nPlease process the attached commercial invoice for our new shipment
from\nChina. We need the vendor details, line items, and total amount extracted\nfor our ERP system. The data 
should be ready by EOD for customs clearance.\n\nThanks!',
                    'image_paths': [
                        '/home/hamzajadoon/Work/PackageX/Agents/email_agent_workflow/./102-ghj-company_png.rf.320a2
b55c06bdd1b43ad1404ed4db231.jpg'
                    ],
                    'document_type': 'invoice'
                },
                'id': 'call_7E4Do97RVGwJpyvcFzIrHWbK',
                'type': 'tool_call'
            }
        ],
        usage_metadata={
            'input_tokens': 1438,
            'output_tokens': 134,
            'total_tokens': 1572,
            'input_token_details': {'audio': 0, 'cache_read': 0},
            'output_token_details': {'audio': 0, 'reasoning': 0}
        }
    ),
    ToolMessage(
        content='Successfully transferred to document_processor_agent',
        name='transfer_to_document_processor_agent',
        id='c89bc2eb-eb09-4b6b-9d02-f69228a04b03',
        tool_call_id='call_7E4Do97RVGwJpyvcFzIrHWbK'
    ),
    AIMessage(
        content='',
        additional_kwargs={
            'tool_calls': [
                {
                    'id': 'call_433geOrqQUD7Zxb3

{
    "email_summary": "Request to process attached commercial invoice for new shipment from China for ERP system.",
    "processing_intent": "data_extraction_requested",
    "tool_outputs": {
        "102-ghj-company_png.rf.320a2b55c06bdd1b43ad1404ed4db231.jpg": {
            "message": "Bill of Lading Inference created successfully",
            "data": {
                "object": "bill_of_lading_inference",
                "id": "inf_bol_65bnrg26wKEASqGSZQzqxH",
                "organization_id": "org_nV7B8MDXzyEYtKTbGpEe5h",
                "status": "inferred",
                "image_url": 
"https://storage.googleapis.com/px-platform-sandbox-scans/img_9cdDYyFaXStPQkdwyEvnqJ.jpeg",
                "raw_text": "wqer company\n6541 address\nqwer 12345\nBill To\nghj company\n8777 address\nhjk 
12345\nDESCRIPTION\ndescription 12345\nTerms & Conditions\nPayment is due within 15 days\nINVOICE\nInvoice 
#\n102\nInvoice Date\n09/06/2022\nAMOUNT\n54,212.00\nTOTAL\n54,212.00",
                "inference": {
                    "sender": {
                        "business": "Wqer Company",
                        "address": {
                            "uuid": "h7yCQ3N9mRoyTnz9MJrXXz",
                            "object": "address",
                            "postal_code": "12345",
                            "formatted_address": "12345",
                            "textarea": " 12345",
                            "verified": false,
                            "hash": "h7yCQ3N9mRoyTnz9MJrXXz"
                        }
                    },
                    "transit": {
                        "address": {
                            "object": "address",
                            "verified": false
                        }
                    },
                    "recipient": {
                        "business": "Ghj Company",
                        "address": {
                            "uuid": "1RNPc47xDie5B8EtHfjrvv",
                            "object": "address",
                            "line1": "8777 address",
                            "city": "hjk",
                            "postal_code": "12345",
                            "formatted_address": "8777 address, hjk 12345",
                            "textarea": "8777 address, hjk 12345",
                            "verified": false,
                            "hash": "1RNPc47xDie5B8EtHfjrvv"
                        }
                    },
                    "logistics_attributes": {
                        "invoice_number": "102",
                        "shipping_id": "54,212.00",
                        "shipping_date": {
                            "year": 2022,
                            "month": 6,
                            "day": 9
                        }
                    },
                    "custom_attributes": {},
                    "tables": [
                        [
                            {
                                "AMOUNT": "54,212.00",
                                "DESCRIPTION": "description 12345"
                            }
                        ]
                    ]
                },
                "matches": {
                    "manifests": [],
                    "items": []
                },
                "options": {
                    "match": {},
                    "search_threshold": {
                        "items": 0.6,
                        "manifests": 0.6
                    }
                },
                "metadata": {},
                "created_at": 1749634004,
                "updated_at": 1749634005,
                "updated_by": "key_pHFP3eevnhNq4FRLcRtgio",
                "checksum": "77aee6ed27533e3c151f2d695ffe8762"
            },
            "status": 201,
            "errors": []
        }
    }
}

-------------------------------------------------------------
